In [ ]:
import numpy as np
import scipy as sp

from plotter import Plotter

In [ ]:
write = True

In [ ]:
study = "Becker"
file_path = "../data/safs417nc3_er.s.0.75.WLD.200.savd.dat"
data = np.loadtxt(file_path)
n_materials = np.size(data,0)
lons = data[:n_materials,0]
lats = data[:n_materials,1]
cijs = data[:n_materials,3:24]

In [ ]:
betas = np.load(f"Data/{study}/betas_{study}.npy")
dp_m1 = sp.io.loadmat(f"Data/{study}/decomp_percentages_{study}.mat")["DP_MS_axes"]
dp_m2 = sp.io.loadmat(f"Data/{study}/decomp_percentages_{study}.mat")["DP_MS_axes_X3_stiff"]
dp_m3 = sp.io.loadmat(f"Data/{study}/decomp_percentages_{study}.mat")["DP_I"]
dp_s1 = np.loadtxt(f"Data/Becker/tens.200.dat")
dp_s1[:,0] = 1 - dp_s1[:,0]

In [ ]:
def cumulation(decomp_percentages):
    n = decomp_percentages.shape[1]
    cumulative_percentages = np.copy(decomp_percentages)
    for i in range(1,n):
        cumulative_percentages[:,i] += cumulative_percentages[:,i-1]
    return cumulative_percentages

def get_alpha(p_cumulative):
    return np.rad2deg(np.arcsin(1 - p_cumulative)) 
        
cumulative_dp_m1 = cumulation(dp_m1)
cumulative_dp_m2 = cumulation(dp_m2)
cumulative_dp_m3 = cumulation(dp_m3)

alphas_m1 = np.zeros(np.shape(dp_m1))
alphas_m2 = np.zeros(np.shape(dp_m2))
alphas_m3 = np.zeros(np.shape(dp_m3))
for i in range(dp_m1.shape[1]):
    alphas_m1[:,i] = get_alpha(cumulative_dp_m1[:,i])
    alphas_m2[:,i] = get_alpha(cumulative_dp_m2[:,i])
    alphas_m3[:,i] = get_alpha(cumulative_dp_m3[:,i])

In [ ]:
symmetry_classes = ["ISO","XISO","TET","ORTH","MONO"]

In [ ]:
# alpha vs beta cmaps

cmaps1 = [('RdYlGn', False, None, None),
          (   'jet', False,  0.0, 1.20),
          (  'BrBG', False,  0.0, 1.20),
          (  'PiYG', False,  0.0, 0.60),
          (  'PRGn', False,  0.0, 0.60)]

rcmaps1 = [( 'Reds', False, None, None),
           ( 'Reds', False,  0.0, 0.02),
           ( 'Reds', False,  0.0, 0.02),
           ( 'Reds', False,  0.0, 0.10),
           ( 'Reds', False,  0.0, 0.60)]

In [ ]:
plot = Plotter(lons, lats, central_longitude=180, write=write)

In [ ]:
# alpha vs beta

for i, sigma in enumerate(symmetry_classes):
    x = betas[i,:]
    y = alphas_m1[:,i]
    xlabel = fr"$\beta_{{{sigma}}}$ (°)"    
    ylabel = fr"$\alpha_{{{sigma}}}$ (°)"   
    filename = f"alpha_beta_{sigma}_{study}"
    plot.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmaps1[i], residual=True, rcmaps=rcmaps1[i], trend=True, aspect="equal", filename=filename)

In [ ]:
# sigma_percentage_sav2decompose vs sigma_percentage_MSAT cmaps

cmaps2 = [['RdYlGn',  True, None, None],
          [   'jet',  True,  0.0, 9.00],
          [  'BrBG',  True,  0.0, 0.05],
          [  'PiYG',  True,  0.0, 2.00],
          [  'PRGn',  True,  0.0, 0.20]]

rcmaps2 = [['seismic', False,   None,  None],
           [   'Reds', False,  0.000, 1.000],
           ['seismic', False, -0.025, 0.025],
           ['seismic', False, -0.250, 0.250],
           ['seismic', False, -0.200, 0.200]]

In [ ]:
# sigma_percentage_sav2decompose vs sigma_percentage_MSAT

for i, sigma in enumerate(symmetry_classes):
    
    sigma2 = sigma
    if sigma=="XISO": sigma2 = "HEX"
     
    x = dp_m1[:,i] * 100    
    y = dp_s1[:,i] * 100
    xlabel = f"{sigma2}_percent_MSAT"
    ylabel = f"{sigma2}_percent_sav2decompose"
    filename = f"percentage_sav2decompose_percentage_MSAT_{sigma2}_{study}"
    plot.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmaps2[i], residual=True, rcmaps=rcmaps2[i], trend=True, aspect="equal", filename=filename)

In [ ]:
# sigma_percentage_sav2decompose vs sigma_percentage_MSAT - test tensor

test_tensor = 9179
plot_highlight = Plotter(lons, lats, central_longitude=180, write=write, highlight=test_tensor)

for i in [1,3]:
    
    sigma = symmetry_classes[i]
    sigma2 = sigma
    if sigma=="XISO": sigma2 = "HEX"
     
    x = dp_m1[:,i] * 100    
    y = dp_s1[:,i] * 100
    xlabel = f"{sigma2}_percent_MSAT"
    ylabel = f"{sigma2}_percent_sav2decompose"
    
    filename = f"percentage_sav2decompose_percentage_MSAT_{sigma2}_{study}_highlighted"
    plot_highlight.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmaps2[i], residual=True, rcmaps=rcmaps2[i], trend=True, aspect="equal", filename=filename)
    
    filename = f"percentage_sav2decompose_percentage_MSAT_correlation_{sigma2}_{study}_highlighted"
    plot_highlight.correlation_plot(x, y, xlabel, ylabel, filename=filename)

In [ ]:
# sigma_percentage_MSAT vs beta_sigma

for i in [1,3]:
    
    sigma = symmetry_classes[i]
    sigma2 = sigma
    if sigma=="XISO": sigma2 = "HEX"

    x = betas[i,:]
    y = dp_m1[:,i] * 100
    xlabel = fr"$\beta_{{{sigma}}}$ (°)"    
    ylabel = f"{sigma2}_percentage_MSAT"
    filename = f"{sigma2}_percentage_MSAT_beta_{sigma}_{study}"
    plot.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmaps1[i], cmaps2=cmaps2[i], residual=False, filename=filename)

In [ ]:
# HEX_percentage_MSAT vs beta_XISO - test tensor

test_tensor = 12794
plot_highlight = Plotter(lons, lats, central_longitude=180, write=write, highlight=test_tensor)

for i in [1]:
    
    sigma = symmetry_classes[i]
    sigma2 = sigma
    if sigma=="XISO": sigma2 = "HEX"
     
    x = betas[i,:]
    y = dp_m1[:,i] * 100
    xlabel = fr"$\beta_{{{sigma}}}$ (°)"    
    ylabel = f"{sigma2}_percentage_MSAT"
    filename = f"{sigma2}_percentage_MSAT_beta_{sigma}_{study}_highlighted"
    plot_highlight.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmaps1[i], cmaps2=cmaps2[i], residual=False, filename=filename)

In [ ]:
# alpha variates vs beta For XISO

rcmaps = [( 'Reds', False,  0.0, 0.02),
          ( 'Reds', False,  0.0, 3.00),
          ( 'Reds', False,  0.0, 4.00)]

# alpha_MS_axes vs beta
for i, sigma in enumerate(["XISO"], start=1):
    x = betas[i,:]
    y = alphas_m1[:,i]
    xlabel = fr"$\beta_{{{sigma}}}$ (°)"    
    ylabel = fr"$\alpha_{{{sigma}}}$ (°) (MS_axes)"   
    filename = f"alpha_MS_axes_beta_{sigma}_{study}"
    plot.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmaps1[i], residual=True, rcmaps=rcmaps[0], trend=True, aspect="equal", filename=filename)

# alpha_MS_axes_X3_stiff vs beta
for i, sigma in enumerate(["XISO"], start=1):
    x = betas[i,:]
    y = alphas_m2[:,i]
    xlabel = fr"$\beta_{{{sigma}}}$ (°)"    
    ylabel = fr"$\alpha_{{{sigma}}}$ (°) (MS_axes_X3_stiff)"   
    filename = f"alpha_MS_axes_X3_stiff_beta_{sigma}_{study}"
    plot.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmaps1[i], residual=True, rcmaps=rcmaps[1], trend=True, aspect="equal", filename=filename)

# alpha_I vs beta
for i, sigma in enumerate(["XISO"], start=1):
    x = betas[i,:]
    y = alphas_m3[:,i]
    xlabel = fr"$\beta_{{{sigma}}}$ (°)"    
    ylabel = fr"$\alpha_{{{sigma}}}$ (°) (I)"   
    filename = f"alpha_I_beta_{sigma}_{study}"
    plot.correlation_plots(x, y, xlabel, ylabel, cmaps1=cmaps1[i], residual=True, rcmaps=rcmaps[2], trend=True, aspect="equal", filename=filename)